In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_10.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,cellphones and driving\n\nCellphones and drivi...,0.0
1,Thanks Electoral College is a process that tha...,0.0
2,This is one reason why you should join the Uni...,0.0
3,How do you get to where you need to go? Do you...,0.0
4,The development of driverless cars is a good t...,0.0
...,...,...
17983,It is hard to imagine being a scientist at NAS...,0.0
17984,Who doesn't have W CWR now days? It seems like...,0.0
17985,"""Good morning students, I have a very importan...",0.0
17986,"Dear Principle,\n\nI believe that Policy 1 is ...",0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 2025])
Attention Mask size: torch.Size([17988, 2025])
Vocabulary Size: 32950
Max Sequence Length: 2025


In [4]:
model=Sequential()
input_length=2025
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=100,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)



Epoch 1/5


450/450 [==============================] - 301s 665ms/step - loss: 0.0147 - accuracy: 0.9978
Epoch 2/5
450/450 [==============================] - 321s 714ms/step - loss: 5.5975e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 330s 733ms/step - loss: 2.7650e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 326s 724ms/step - loss: 1.6660e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 327s 726ms/step - loss: 1.1041e-05 - accuracy: 1.0000


In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 27s 235ms/step - loss: 9.1243e-06 - accuracy: 1.0000


[9.124297321250197e-06, 1.0]

In [6]:
model.save("newmodel10.keras")